In [ ]:
!pip3 install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import necessary libraries
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import T5Tokenizer, T5ForConditionalGeneration
from sklearn.model_selection import train_test_split

from datasets import load_metric
import torch
from torch.optim import AdamW
from tqdm.auto import tqdm

In [ ]:
dataset = load_dataset("KaungHtetCho/MedicalQA")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Description', 'Patient', 'Doctor'],
        num_rows: 496792
    })
})

In [ ]:
seed=42

In [ ]:
# Split the dataset into training and test data (90% train, 10% test)
reduce_dataset_split = dataset["train"].train_test_split(test_size=0.01, seed=seed)
train_set = reduce_dataset_split['test']

train_test_split = train_set.train_test_split(test_size=0.1, seed=seed)
test_set = train_test_split['test']

# Further split the test set into validation and test sets (50% validation, 50% test)
val_test_split = test_set.train_test_split(test_size=0.5, seed=seed)
validation_set = val_test_split['train']
test_set = val_test_split['test']

# Update dataset to include these splits
dataset["train"] = train_test_split['train']
dataset["validation"] = validation_set
dataset["test"] = test_set

In [ ]:
dataset["train"]

Dataset({
    features: ['Description', 'Patient', 'Doctor'],
    num_rows: 4471
})

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")

def process_data_to_model_inputs(batch):
    inputs = tokenizer(batch["Patient"], padding="max_length", truncation=True, max_length=512)
    outputs = tokenizer(batch["Doctor"], padding="max_length", truncation=True, max_length=512)

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["labels"] = outputs.input_ids
    return batch

# Tokenize and align the data in the dataset
dataset = dataset.map(process_data_to_model_inputs, batched=True)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/4471 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

Map:   0%|          | 0/249 [00:00<?, ? examples/s]

In [ ]:
def collate_fn(batch):
    # Convert list of dicts to dict of lists
    batch = {key: [d[key] for d in batch] for key in batch[0]}

    # Convert list to tensors
    batch["input_ids"] = torch.tensor(batch["input_ids"]).to(device)
    batch["attention_mask"] = torch.tensor(batch["attention_mask"]).to(device)
    batch["labels"] = torch.tensor(batch["labels"]).to(device)

    return batch

In [ ]:
def validate(model, tokenizer, val_loader):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    with torch.no_grad():  # Disable gradient calculation
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

    average_loss = total_loss / len(val_loader)
    return average_loss

In [ ]:
# Convert to pytorch tensors and create data loaders
train_loader = DataLoader(dataset["train"], batch_size=32, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(dataset["validation"], batch_size=32, collate_fn=collate_fn)
test_loader = DataLoader(dataset["test"], batch_size=32, collate_fn=collate_fn)

In [ ]:
def format_time(seconds):
    """Converts seconds to a string of hours, minutes, and seconds"""
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return f"{int(hours)}h {int(minutes)}m {int(seconds)}s"

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
import time

# Initialize model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Check for GPU availability
model.to(device)

# Prepare optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Define the training loop
num_epoch = 10
prev_val_loss = float('inf')
for epoch in range(num_epoch):  # number of epochs

    start_time = time.time()  # Record the start time of the epoch

    model.train()

    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Calculate the duration of the epoch
    end_time = time.time()
    epoch_duration = end_time - start_time
    formatted_time = format_time(epoch_duration)

    # Compute training and validation loss
    val_loss = validate(model, tokenizer, val_loader)
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f} | Validation Loss: {val_loss:.4f} | Duration: {formatted_time}")

    # Save model and tokenizer after we got better val loss.
    if val_loss < prev_val_loss:
      print(f"{val_loss} < {prev_val_loss}. Saving the model.")
      model.save_pretrained(f"./model_t5")
      tokenizer.save_pretrained(f"./model_t5")
      prev_val_loss = val_loss


Epoch 1, Loss: 3.5580 | Validation Loss: 1.0941 | Duration: 0h 1m 22s'
1.0941301584243774 < inf. Saving the model.
Epoch 2, Loss: 1.2070 | Validation Loss: 1.0385 | Duration: 0h 1m 22s'
1.038529135286808 < 1.0941301584243774. Saving the model.
Epoch 3, Loss: 1.0843 | Validation Loss: 1.0050 | Duration: 0h 1m 22s'
1.004958413541317 < 1.038529135286808. Saving the model.
Epoch 4, Loss: 1.0361 | Validation Loss: 0.9784 | Duration: 0h 1m 22s'
0.9784235805273056 < 1.004958413541317. Saving the model.
Epoch 5, Loss: 1.0037 | Validation Loss: 0.9573 | Duration: 0h 1m 22s'
0.9572854712605476 < 0.9784235805273056. Saving the model.
Epoch 6, Loss: 0.9791 | Validation Loss: 0.9392 | Duration: 0h 1m 22s'
0.9391559734940529 < 0.9572854712605476. Saving the model.
Epoch 7, Loss: 0.9606 | Validation Loss: 0.9231 | Duration: 0h 1m 22s'
0.9231254905462265 < 0.9391559734940529. Saving the model.
Epoch 8, Loss: 0.9429 | Validation Loss: 0.9096 | Duration: 0h 1m 22s'
0.9095947816967964 < 0.923125490546226

In [ ]:
# Save model and tokenizer after each epoch
# model.save_pretrained(f"./model_t5")
# tokenizer.save_pretrained(f"./model_t5")

In [ ]:
model_directory = "model_t5"

In [ ]:
save_path = "/content/drive/MyDrive/Colab_Notebooks/NLP/Project/models/"

In [ ]:
!cp -r $model_directory $save_path

In [ ]:
# # Copy the model if you want to begin the colab from here.
# model_directory = "model_t5"
# save_path = "/content/drive/MyDrive/Colab_Notebooks/NLP/Project/models/" + model_directory
# !cp -r $save_path .

In [ ]:
# Load the model and tokenizer from a specified epoch
device = "cuda" if torch.cuda.is_available() else "cpu"

model_directory = "./model_t5"  # Adjust the path to match where you've saved the model

load_model = T5ForConditionalGeneration.from_pretrained(model_directory)
load_model.to(device)
load_tokenizer = T5Tokenizer.from_pretrained(model_directory)
load_model = load_model.eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from nltk.translate.bleu_score import corpus_bleu

def generate_answers(batch):
    inputs = load_tokenizer(batch["Patient"], return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = load_model.generate(input_ids=inputs["input_ids"].to(device), attention_mask=inputs["attention_mask"].to(device))
    batch["model_output"] = load_tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return batch

results = test_loader.dataset.map(generate_answers, batched=True, batch_size=8)
refs = [[x.split()] for x in results["Doctor"]]
hyp = [x.split() for x in results["model_output"]]

bleu_score = corpus_bleu(refs, hyp)
print("BLEU score:", bleu_score)

Map:   0%|          | 0/24840 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


BLEU score: 0.0005053602162615737


In [ ]:
def answer_question(question):
    load_model.eval()
    with torch.no_grad():
        input_ids = load_tokenizer(question, return_tensors="pt", padding=True, truncation=True, max_length=512).input_ids.to(device)
        outputs = load_model.generate(
            input_ids,
            max_length=512,
            temperature=0.4,
            do_sample=True,
            top_k=20
        )
        answer = load_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

# Example
print(answer_question("I am having a fever, and sore throat. What should I do?"))

Hi,Thanks for your query.I have gone through your query and I can understand your concern.As per your query, I would suggest you to get an x-ray of the chest to rule out the cause.As you have fever, sore throat, sore throat, sore throat, and fever.I would suggest you to take an antihistamine like aceclofenac to rule out the cause of the fever.You should do some swab test to rule out the cause.I hope this helps.Take care.


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
save_path = "/content/drive/MyDrive/Colab_Notebooks/NLP/Project/models"